## Setup ##

In [ ]:
# Import modules #
import copy
import cv2
import ftplib

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import numpy as np
import os
import pickle
import pptk
import SurfRCaT
import time

# Create a working directory #
pth = os.path.realpath('../../../')

## Establish the video to use for the calibration and extract stills ##

In [ ]:
# Establish the video file #
vid = os.path.realpath(pth+os.sep+'JupiterTutorial1.mp4')

# Extract video properties
cap = cv2.VideoCapture(vid)
numFrames = int(cap.get(7))
fps = cap.get(5)
vidLen = int(numFrames/fps)

# Establish desired extraction parameters #
secondsPerFrame = 10
rate = None # Don't need if using a secondsPerFrame #

# Run the frame extraction #
SurfRCaT.getImagery_GetStills(vid,secondsPerFrame,rate,vidLen,fps,pth+os.sep)

## A /_frames directory now exists with extracted frames ##

## Esatblish the extracted frame to use in the calibration ##

In [ ]:
frameNum = 20 # Can choose any of the extracted frames #
img = cv2.imread(os.path.realpath(pth+os.sep+'_frames'+os.sep+'frame'+str(frameNum)+'.png'))
os.mkdir(pth+os.sep+'_products')
cv2.imwrite(pth+os.sep+'_products/calibrationImage.png',img)

## A /_products directory now exists with the chosen calibration image ##

## Establish user input camera parameters  ##

In [ ]:
# Required user inputs #
# NOTE: Can get all of these from Google Earth #
cam_lat = 26.93846
cam_lon = -80.07054
cam_elev = 49.5
cam_az = 350

## Find a lidar dataset to use in remote-GCP extraction ##

In [ ]:
# Find NOAA lidar datasets that cover the correct state or coast #
possibleIDs = SurfRCaT.getLidar_FindPossibleIDs(cam_lat,cam_lon)
print(possibleIDs)

In [ ]:
# Check all possibleIDs to find datasets that cover the camera location - appropIDs #

# NOAA FTP login #
ftp = ftplib.FTP('ftp.coast.noaa.gov',timeout=1000000)
ftp.login('anonymous','anonymous')

# Get a list of all lidar directories in the NOAA FTP. #
ftp.cwd('/pub/DigitalCoast')
dirs = [i for i in ftp.nlst() if 'lidar' in i]
alldirs = []
for ii in dirs:
    ftp.cwd(ii)
    alldirs.append([ii+'/'+i for i in ftp.nlst() if 'geoid' in i])
    ftp.cwd('../')  

# Check each of the possibleIDs to see if it covers the location of the camera #
appropIDs = list()
i = 0
for ID in possibleIDs:
    i = i+1
    perDone = i/len(possibleIDs)
    print(str(perDone*100)+'% done')

    check = SurfRCaT.getLidar_TryID(ftp,alldirs,ID,cam_lat,cam_lon)
    ftp.cwd('/pub/DigitalCoast')

    if check:
        if len(check)>0:       
            appropIDs.append(ID)
            print(appropIDs)

In [ ]:
# Establish the ID of the dataset to use #
useID = 6330

## Download the lidar dataset ##

In [ ]:
# Get the geographic info of the dataset, contained in a shapefile #
sf = SurfRCaT.getLidar_GetShapefile(useID)
print(sf)

In [ ]:
# Calculate the view area of the camera
poly = SurfRCaT.getLidar_CalcViewArea(cam_az,40,1000,cam_lat,cam_lon)
print(poly)

In [ ]:
# Get the dataset tiles that cover the camera area #
tilesKeep = list()
i = 0
for shapeNum in range(0,len(sf)):
    out = SurfRCaT.getLidar_SearchTiles(sf,poly,shapeNum,cam_lat,cam_lon)
    if out:
        tilesKeep.append(out)
    i = i+1
    perDone = i/len(sf)
    print(str(perDone*100)+'% done')
print(tilesKeep)

In [ ]:
# Download the data or use pre-downloaded #
#i = 0
#lidarDat = np.empty([0,3])
#for thisFile in tilesKeep:
#
#    lidarXYZsmall = SurfRCaT.getLidar_Download(thisFile,useID,cam_lat,cam_lon)
#
#    lidarDat = np.append(lidarDat,lidarXYZsmall,axis=0)
#
#    i = i+1
#    perDone = i/len(tilesKeep)
#    print(str(perDone*100)+'% done')


# Use pre-downloaded lidar data #    
with open(pth+os.sep+'demoLidarFile.pkl','rb') as f:
    lidarDat = pickle.load(f)

In [ ]:
# Create a point cloud with local coordinates relative to the camera location #
pc = SurfRCaT.getLidar_CreatePC(lidarDat,cam_lat,cam_lon)
print(pc)

## Pick GCPs in the image ##

In [ ]:
im = plt.imread(pth+os.sep+'_products/calibrationImage.png')
plt.imshow(im)
GCPs_im = plt.ginput(4)
GCPs_im = np.array(np.array(GCPs_im))
print(GCPs_im)

## Pick GCPs in the lidar ##

In [ ]:
# Launch the point cloud viewer 
v = pptk.viewer(pc,pc.iloc[:,2])
v.set(point_size=0.1,theta=-25,phi=0,lookat=[0,0,20],color_map_scale=[-1,10],r=0)

In [ ]:
# Do the GCP picking #
iGCPs_lidar = np.empty([0,1])
while 1<2: # Continuously test to see if viewer window is open #
    try:
        test = v.get('curr_attribute_id')
        a = v.get('selected')
        if len(a)>0:
            a = int(a)
            iGCPs_lidar = np.vstack([iGCPs_lidar,a])
        else:
            a = 0
        del a
        time.sleep(2)
    except ConnectionRefusedError:
        break
iGCPs_lidar = np.unique(iGCPs_lidar).astype(int)
print(iGCPs_lidar)

In [ ]:
# Get the GCP coordinates #
GCPs_lidar = np.empty([0,3])
for i in sorted(iGCPs_lidar):
    GCPs_lidar = np.vstack((GCPs_lidar,pc.iloc[i,:]))
iss = np.argsort(GCPs_lidar[:,2])
GCPs_lidar = np.flipud(GCPs_lidar[iss,:])
print(GCPs_lidar)

## Perform the calibration ##

In [ ]:
# Estimate instrinsic parmeters from the image characteristics #
cam_f,cam_x0,cam_y0 = SurfRCaT.calibrate_GetInitialApprox_IOPs(cv2.imread(pth+os.sep+'_products/calibrationImage.png'))

In [ ]:
# Convert rotation systems from azimuth, tilt, swing to omega, phi, kappa #
# Note the following assumptions: tilt = 80, swing = 180 #
cam_omega,cam_phi,cam_kappa = SurfRCaT.calibrate_GetInitialApprox_ats2opk(cam_az,80,180)

In [ ]:
# Perform the calibration in a two-step optimization #
initApprox = np.array([cam_omega,cam_phi,cam_kappa,0,0,cam_elev,cam_f,cam_x0,cam_y0])
freeVec1 = np.array([0,0,0,1,1,1,0,0,0])
freeVec2 = np.array([1,1,1,0,0,0,1,1,1])
calibVals1,se1 = SurfRCaT.calibrate_PerformCalibration(initApprox,freeVec1,GCPs_im,GCPs_lidar)
updatedApprox = copy.copy(calibVals1)
calibVals2,se2 = SurfRCaT.calibrate_PerformCalibration(updatedApprox,freeVec2,GCPs_im,GCPs_lidar)

In [ ]:
# Check the calibration by reprojecting the GCPs onto the image #
u_reproj,v_reproj = SurfRCaT.calibrate_CalcReprojPos(GCPs_lidar,calibVals2)

fig,ax = plt.subplots(1)
ax.imshow(im)
cols = ['r','g','b','c']
for i in range(len(GCPs_im)):
    ax.plot(GCPs_im[i,0],GCPs_im[i,1],'o',markeredgecolor=cols[i],markerfacecolor='w')
    ax.plot(u_reproj[i],v_reproj[i],'x',color=cols[i])
fig.show()

## Rectify the image ##

In [ ]:
# Establsih the real-world rectification grid #
xmin = -200
xmax = 200
dx = 0.5
ymin = 250
ymax = 900
dy = 0.5
z = 0.1 # Tide level, can get from NOAA #
grid = [xmin,xmax,dx,ymin,ymax,dy,z]

In [ ]:
# Perform the rectification #
im_rectif,extents = SurfRCaT.rectify_RectifyImage(calibVals2,im,grid)

In [ ]:
# Plot the rectified image #
fig,ax = plt.subplots(1)
ax.imshow(im_rectif,extent=extents,interpolation='bilinear')
ax.set_xlabel('Local x (m)')
ax.set_ylabel('Local y (m)')
ax.axis('equal')
fig.show()